In [10]:
# File: interactive_paint_with_color_selection.py
import cv2
import mediapipe as mp
import numpy as np
import math

# Inicialización de Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Configuración de colores
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (0, 255, 255)]  # Rojo, Verde, Azul, Amarillo
current_color = colors[0]
brush_size = 10
erase_mode = False

# Variables del menú y pintura
menu_open = False
menu_width = 100
drawing = False
canvas = None  # Para almacenar lo dibujado
dominant_hand = None  # Será "Left" o "Right"

# Funciones auxiliares
def draw_menu(frame, menu_open):
    """Dibuja el menú lateral izquierdo."""
    if menu_open:
        cv2.rectangle(frame, (0, 0), (menu_width, frame.shape[0]), (50, 50, 50), -1)
        for i, color in enumerate(colors):
            cv2.circle(frame, (50, 100 + i * 100), 30, color, -1)
    else:
        cv2.arrowedLine(frame, (10, frame.shape[0] // 2), (50, frame.shape[0] // 2), (50, 50, 50), 5)

def count_fingers(hand_landmarks):
    """Cuenta los dedos levantados basado en los puntos clave."""
    tips = [
        mp_hands.HandLandmark.THUMB_TIP,
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP,
    ]
    mcp = [
        mp_hands.HandLandmark.THUMB_IP,
        mp_hands.HandLandmark.INDEX_FINGER_MCP,
        mp_hands.HandLandmark.MIDDLE_FINGER_MCP,
        mp_hands.HandLandmark.RING_FINGER_MCP,
        mp_hands.HandLandmark.PINKY_MCP,
    ]

    count = 0
    for tip, base in zip(tips, mcp):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[base].y:
            count += 1
    return count

def distance_between_points(p1, p2, frame_width, frame_height):
    """Calcula la distancia euclidiana entre dos puntos escalados por el tamaño del frame."""
    x1, y1 = int(p1.x * frame_width), int(p1.y * frame_height)
    x2, y2 = int(p2.x * frame_width), int(p2.y * frame_height)
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def detect_dominant_hand(results):
    """Detecta si la mano es izquierda o derecha."""
    if results.multi_handedness:
        for hand in results.multi_handedness:
            return hand.classification[0].label  # "Left" o "Right"
    return None

# Iniciar cámara
cap = cv2.VideoCapture(0)
calibrating = True  # Modo calibración

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if canvas is None:
        canvas = np.zeros_like(frame)  # Inicializar el lienzo

    frame = cv2.flip(frame, 1)  # Flip horizontal para parecer espejo
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    frame_height, frame_width, _ = frame.shape

    if calibrating:
        # Mostrar mensaje para calibrar
        cv2.putText(frame, "Muestra tu mano dominante", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Detectar la mano dominante
        if results.multi_hand_landmarks:
            dominant_hand = detect_dominant_hand(results)
            if dominant_hand:
                calibrating = False  # Salir de calibración

    else:
        # Detección de manos (solo rastrear la dominante)
        if results.multi_handedness and results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                handedness = results.multi_handedness[i].classification[0].label
                if handedness != dominant_hand:
                    continue  # Ignorar manos que no sean la dominante

                # Coordenadas del índice
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger.x * frame_width), int(index_finger.y * frame_height)

                # Detectar interacción con el menú
                wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame_width)
                wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame_height)
                if wrist_x < menu_width:
                    menu_open = True
                else:
                    menu_open = False

                if menu_open:
                    # Cambiar color si se toca un círculo de color
                    for i, color in enumerate(colors):
                        cx, cy = 50, 100 + i * 100
                        if math.hypot(x - cx, y - cy) < 30:
                            current_color = color
                else:
                    # Dibujar o borrar según la posición del índice
                    if drawing:
                        if erase_mode:
                            cv2.circle(canvas, (x, y), brush_size, (0, 0, 0), -1)  # Borrar
                        else:
                            cv2.circle(canvas, (x, y), brush_size, current_color, -1)  # Pintar

                # Mostrar el color del índice
                cv2.circle(frame, (x, y), 10, current_color, -1)  # Colorear la punta del índice

    # Combinar el lienzo con el frame
    frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Dibujar el menú
    draw_menu(frame, menu_open)

    # Mostrar el frame
    cv2.imshow("Paint Interactivo", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Presionar ESC para salir
        break
    elif key & 0xFF == ord('d'):  # Activar/desactivar dibujo
        drawing = not drawing

cap.release()
cv2.destroyAllWindows()
